In [ ]:
# !pip install git+https://github.com/alec-tschantz/pybrid.git

In [ ]:
import logging
import torch
from pybrid.scripts import main
from pybrid.config import cfg

In [ ]:
cfg.exp.log_dir = "results/predcoding"
cfg.model.train_amortised = False 

seeds = [0, 1, 2]
for seed in seeds:
    cfg.exp.seed = seed
    main(cfg)